## **1. Introducció**

League of Legends (LoL) és un videojoc del gènere Multiplayer Online Battle Arena (MOBA) desenvolupat i publicat per Riot Games i és el videojoc més popular d'aquest gènere. El joc consisteix a jugar partides autocontingudes, fent referència al fet que jugar una partida no afecta directament les pròximes partides, entre dos equips diferents (Blau i Vermell) que han de destruir la base (nexe) de l'equip contrari. Aquesta és l'única manera de guanyar la partida, assumint que cap equip es rendeixi, ja que és impossible quedar empatats. 

Ara, es tracta d'un joc d'estratègia i, per tant, hi ha diferents mecàniques del joc que donen avantatge a l'equip que l'aconsegueixi o l'aprofiti millor. En conseqüència, aquestes mecàniques poden ser la clau per a guanyar una partida, tot i que hi ha moltes més variables a tenir en compte.

L'objectiu d'aquest notebook és analitzar el conjunt de dades o dataset de Partides Competitives (SOLO-Q) de League of Legends i veure si, amb les dades disponibles, es pot entrenar un model de predicció capaç de predir quin serà l'equip que guanyarà la partida. S'ha de tenir en compte que aquest dataset és de l'any 2019 i, per tant, pot no reflectir amb exactitud l'estat actual del joc i es tindrà en compte a les conclusions.

## **2. Explicació del _Gameplay_**

Per a poder entendre i justificar les característiques del dataset, cal entendre algunes de les mecàniques d'una partida classificatòria de League of Legends (versió 2019).

#### **Objectiu principal**
L'objectiu principal de la partida és el **Nexe** enemic, situat en la seva base: si el teu Nexe és destruït, perds la partida; si destrueixes el Nexe enemic, guanyes. El nexe està protegit per dues Torretes interiors que cal destruir primer. Tota l'estratègia del joc cau en jugar al voltant dels objectius. Cada jugador de l'equip escull un campió diferent (personatge amb diferents atacs i habilitats).

#### **Estructures defensives**

* **Inhibidors**: cada equip té tres Inhibidors, un per línia. Es troben la base de l'equip i cada una és protegida per una Torreta que ha de ser destruïda. En destruir-ne un, l'equip passa a generar minions millorats només en aquella línia, facilitant l'avanç per ella. Mínim un ha de ser destruït per accedir al Nexe.

* **Torretes**: poden atacar als jugadors enemics si peguen a l'enemic en el seu radi. Protegeixen les línies i la base. Cada línia en té tres, tenint en compte la de l'inhibidor, i han de ser destruïdes en ordre per a cada línia. La primera Torreta destruïda atorga una bonificació d'or.

#### **Monstres èpics**

* **Herald de l'escletxa**: apareix en el minut 10:00 de la partida. L'equip que el derroti pot aconseguir un objecte que invoca l'Herald, que ajuda a destruir les estructures enemigues d'una línia.

* **Dracs**: apareixen reiteradament durant el transcurs de la partida (canviant d'element) fins que un equip n'aconsegueixi quatre. Depenent de l'element, augmenta estadístiques (dany, resistències, velocitat, etc.) de l'equip.

* **Drac ancià**: després de que un equip hagi matat a quatre dracs, la seva aparició es canvia per la del drac ancià, aquest drac atorga estadistiques que tenen un impacte major sobre les estadistiques de l'equip.

* **Baró Nashor**: apareix en el minut 20:00 de la partida. Atorga una millora temporal que incrementa la força de l'equip i dels minions, facilitant l'avanç per les línies i la base enemiga.<br><br>

Amb els diferents objectius, farmeant minions i monstres de la jungla i aconseguint morts enemigues (la primera mort atorga una bonificació d'or) s'atorga or i experiència a l'equip que els aconsegueix. També es pot col·locar un element de visió al mapa que s'anomena "Ward" o Guarda, en català. El guarda permet veure si un enemic es mou d'una línia a una altra o si s'estan fent un objectiu, és a dir, atorga visió per a tenir un control sobre les decisions de l'equip rival. Es pot treure la visió de l'equip rival, atorgant una mica d'or. L'or permet la compra d'objectes que milloren les estadístiques dels campions, mentre que l'experiència permet pujar de nivell les habilitats, facilitant l'obtenció de més objectius i, finalment, la victòria.

Resumint com funciona la partida: Els jugadors apareixen en la base en el minut 0:00, anant a les seves línies per a jugar. Durant el transcurs, intentaran avançar per les línies, destruint les torretes, l'inhibidor i finalment, el nexe i les seves torretes. També aniran matant als minions, monstres menors i campions enemics, alhora que intenten aconseguir els monstres. Podran comprar amb l'or aconseguit per enfortir-se i col·locar visió pel mapa per tenir un control més elevat en la partida. Cada cop que un jugador mor, reapareix en un temps determinat pel seu nivell. Per tant, a final de partida, tardaran més en reaparèixer.



<p align="center">
  <img src="models/Mapa League of Legends simplificat.png" width="750">
</p>


## **3. Dependències**

En aquest apartat es troben totes les llibreries que s'han utilitzat en l'anàlisi del nostre dataset i, per tant, es requereixen per poder executar correctament el nostre notebook:

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.svm import SVC
from sklearn.linear_model import RidgeClassifier, LogisticRegression
from sklearn.ensemble import RandomForestClassifier, HistGradientBoostingClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, roc_auc_score, roc_curve
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.pipeline import Pipeline
from sklearn.pipeline import make_pipeline
from sklearn.compose import make_column_transformer
from sklearn.model_selection import cross_val_score

## **4. Anàlisi exploratori del dataset**